# NumPyNMF Repository Quick Start

## Setup

In [ ]:
# Clone the repository
!git clone https://github.com/XavierSpycy/NumPyNMF.git
# Switch to the NumPyNMF directory
%cd NumPyNMF

## Play with Our Existing Methods

### Run the Script in the Terminal

In [ ]:
!python run.py --dataset 'ORL' --reduce 1 --max_iter 500

### Run the Script in the Notebook

In [ ]:
from algorithm.pipeline import Pipeline

pipeline = Pipeline(nmf='L1NormRegularizedNMF', # Options: 'L2NormNMF', 'KLDivergenceNMF', 'ISDivergenceNMF', 'L21NormNMF', 'HSCostNMF', 'L1NormRegularizedNMF', 'CappedNormNMF', 'CauchyNMF'
                    dataset='YaleB', # Options: 'ORL', 'YaleB'
                    reduce=3, # ORL: 1, YaleB: 3
                    noise_type='salt_and_pepper', # Options: 'uniform', 'gaussian', 'laplacian', 'salt_and_pepper', 'block'
                    noise_level=0.08, # Uniform, Gassian, Laplacian: [.1, .3], Salt and Pepper: [.02, .10], Block: [10, 20]
                    random_state=99, # 0, 42, 99, 512, 3407 in our experiments
                    scaler='MinMax') # Options: 'MinMax', 'Standard'

# Run the pipeline
pipeline.execute(max_iter=500, verbose=True) # Parameters: max_iter: int, convergence_trend: bool, matrix_size: bool, verbose: bool
pipeline.evaluate(idx=9, imshow=True) # Parameters: idx: int, imshow: bool

## Develop Your Customized NMF

In [ ]:
import numpy as np
from algorithm.nmf import BasicNMF
from algorithm.pipeline import Pipeline

class ExampleNMF(BasicNMF):
    name = 'Example'
    # If you would like to design your customized NMF algorithm, you can inherit the BasicNMF class
    # and override the matrix_init and update methods
    def matrix_init(self, X, n_components, random_state=None):
        # How to employ the random_state depends on the algorithm
        # For example, you can use np.random.seed(random_state)
        n_features = X.shape[0]
        n_samples = X.shape[1]
        D = np.random.normal(loc=0, scale=1, size=(n_features, n_components))
        R = np.random.uniform(low=0, high=1, size=(n_components, n_samples))
        return D, R # Make sure to return D, R

    def update(self, X, **kwargs):
        # Take L2-norm based NMF as an example
        # Get the parameters from kwargs as you need
        threshold = kwargs.get('threshold', 1e-6)
        epsilon = kwargs.get('epsilon', 1e-7)
        # Multiplicative update rule for D and R matrices
        self.D *= np.dot(X, self.R.T) / (np.dot(np.dot(self.D, self.R), self.R.T) + epsilon)
        self.R *= np.dot(self.D.T, X) / (np.dot(np.dot(self.D.T, self.D), self.R) + epsilon)
        # Calculate the loss function
        loss = np.linalg.norm(X - np.dot(self.D, self.R), 'fro') ** 2
        self.loss_list.append(loss)
        # Calculate L2-norm based errors for convergence
        e_D = np.sqrt(np.sum((self.D - self.D_prev) ** 2, axis=(0, 1))) / self.D.size
        e_R = np.sqrt(np.sum((self.R - self.R_prev) ** 2, axis=(0, 1))) / self.R.size
        # Update previous matrices for next iteration
        self.D_prev, self.R_prev = self.D.copy(), self.R.copy()
        return (e_D < threshold and e_R < threshold) # Return True if converged, False otherwise

# Run the pipeline
pipeline = Pipeline(nmf=ExampleNMF(),
                    dataset='YaleB',
                    reduce=3,
                    noise_type='salt_and_pepper',
                    noise_level=0.08,
                    random_state=99,
                    scaler='MinMax')

# Run the pipeline
# Due to the initialization of D and R matrices, 
# the performance of the ExampleNMF algorithm may be worse than our built-in algorithm.
pipeline.execute(max_iter=500, verbose=True)
pipeline.evaluate(idx=9, imshow=False)

## Intensive Experiments

In [ ]:
from algorithm.pipeline import Experiment, summary

model_name = 'L1NormRegularizedNMF'

In [ ]:
exp = Experiment()
# Once you build the data container
# You can choose an NMF algorithm and execute the experiment
exp.choose(model_name)
# This step is very time-consuming, please be patient.
# If you achieve a better performance, congratulations! 
# You can share your results with us.
# Similarly, you can replace 'L1NormRegularizedNMF' with other your customized NMF algorithm
exp.execute()

In [ ]:
summary(f'{model_name.split("NMF")[0]}_log.csv')